In [23]:
import pandas as pd
from process import run_ols
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from scipy.stats import norm
from MLE import MLE
from process import run_ols

In [24]:
players = [] # array of dataframes where players[i] is the i+1th player's data
for i in range(1000):
    file = "../data/player_data_one_hot/student_" + str(i + 1) + "_one_hot.csv"
    players.append(pd.read_csv(file))

In [25]:
# maybe find trend between all player's m, n, e, find which covariates affect this sport and
# make a model given expected score and covariates, obtain m, n, and e?

# Step 1: obtain all data from singular_one_hot
soh = pd.read_csv("../data/singular_one_hot.csv")
morning_oh = soh[soh["x0_morning"] == 1.0]
noon_oh = soh[soh["x0_noon"] == 1.0]
evening_oh = soh[soh["x0_evening"] == 1.0]

In [28]:
# Step 2: run regression on the data for morning
formula = "score ~ fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + west_coast + early_bird + night_owl + frosh + junior + senior + soph"
morning_result = run_ols(formula, morning_oh)

In [29]:
# Step 3: run regression on the data for noon
formula = "score ~ fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + west_coast + early_bird + night_owl + frosh + junior + senior + soph"
noon_result = run_ols(formula, noon_oh)

In [30]:
# Step 4: run regression on the data for evening
formula = "score ~ fitness_score + extra_strategy_sessions_attended + hours_of_sleep + meals + east_coast + west_coast + early_bird + night_owl + frosh + junior + senior + soph"
evening_result = run_ols(formula, evening_oh)

In [33]:
# Step 5: prediction function
keys = {"fitness_score", "extra_strategy_sessions_attended", "hours_of_sleep", "meals", "east_coast", "west_coast", "early_bird", "night_owl", "frosh", "junior", "senior", "soph"}
def predict_score(type, player):
    ols_prediction_model = None
    player_avg = player.mean(axis=0)
    if type == "morning":
        ols_prediction_model = morning_result.params
    elif type == "noon":
        ols_prediction_model = noon_result.params
    elif type == "evening":
        ols_prediction_model = evening_result.params
    else:
        print("this is illegal")

    # return prediction
    return ols_prediction_model["Intercept"] + np.sum([ols_prediction_model[key] * player_avg[key] for key in keys])

print(predict_score("morning", players[0]))

33.31234299566217


/var/folders/h3/v6p39mmn4snd_1b2tm2knxnw0000gn/T/ipykernel_51415/788651735.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  player_avg = player.mean(axis=0)


In [3]:
total_score = np.zeros((3, 1000))
num_score = np.zeros((3, 1000))

for i in range(1000):
    player = players[i]
    # replaced by predict_score function
    for _, row in player.iterrows():
        if row["game_scheduled"] == "morning":
            total_score[0][i] += row["score"]
            num_score[0][i] += 1
        elif row["game_scheduled"] == "noon":
            total_score[1][i] += row["score"]
            num_score[1][i] += 1
        elif row["game_scheduled"] == "evening":
            total_score[2][i] += row["score"]
            num_score[2][i] += 1

avg_score = np.divide(total_score, num_score, out=np.zeros_like(total_score), where=num_score!=0)

In [4]:
# avg_score[0] = "m", avg_score[1] = "n", avg_score[2] = "e"
# huge problem with expected_score as some of the scores are 0
# maybe find trend between all player's m, n, e, find which covariates affect this sport and make a model given expected score and covariates, obtain m, n, and e?
expected_score = np.sum(avg_score, axis=0) / 3

In [5]:
# determine f(x, y)
mle = MLE("../data/previous_season_results.csv")
sigma = mle.calculate_MLE() # obtain from MLE
print(sigma)
def winProb(score_1, score_2):
    return 1 - norm(loc = score_1 - score_2 , scale = sigma).cdf(0)

10.828673870065314


In [18]:
# to maximize E[win], we should choose the 10 highest players
top_athletes = np.argpartition(expected_score, -10)[-10:] + 1
print("these are our top athletes:", top_athletes)

# team score
top_scores = expected_score[top_athletes - 1]
top_team_score = np.average(top_scores)
print("our team score is: ", top_team_score)

# expected number of wins
matchup = pd.read_csv("../data/season_match_up.csv")
matchup = matchup.to_numpy()[0, 1:] # 20 games
E_win = np.sum([winProb(top_team_score, matchup_score) for matchup_score in matchup])
print("Expected number of wins: ", E_win)

these are our top athletes: [268 725 790  89 138 791 631 785 521 129]
our team score is:  75.33198809523809
Expected number of wins:  11.375058670272534
